In [1]:
%pip install torch torchvision clip-by-openai flask flask-cors numpy pillow requests

  Using cached clip_by_openai-1.1-py3-none-any.whl.metadata (369 bytes)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached clip_by_openai-1.0.1-py3-none-any.whl.metadata (407 bytes)
  Using cached clip_by_openai-0.1.1.5-py3-none-any.whl.metadata (8.6 kB)
  Using cached clip_by_openai-0.1.1.4-py3-none-any.whl.metadata (8.6 kB)
  Using cached clip_by_openai-0.1.1.3-py3-none-any.whl.metadata (8.7 kB)
  Using cached clip_by_openai-0.1.1.2-py3-none-any.whl.metadata (9.0 kB)
  Using cached clip_by_openai-0.1.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached clip_by_openai-0.1.0-py3-none-any.whl.metadata (9.0 kB)
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might n

ERROR: Cannot install torch, torchvision==0.17.1, torchvision==0.17.2, torchvision==0.18.0, torchvision==0.18.1, torchvision==0.19.0, torchvision==0.19.1, torchvision==0.20.0, torchvision==0.20.1, torchvision==0.21.0 and torchvision==0.22.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
import sys
!{sys.executable} -m pip install torch


In [3]:
import sys
!{sys.executable} -m pip install torchvision

In [4]:
%pip uninstall clip-by-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\zainabfr\appdata\local\temp\pip-req-build-v7lx0zrs
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
Using cached ftfy-6.3.1-py3-none-any.whl (44 kB)
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369570 sha256=fa9faeb5ddff0df4e90fd0c415d50fc2d6243a4af0814c1913ceeb6e7074e305
  Stored in directory: C:\Users\ZainabFR\AppData\Local\Temp\pip-ephem-wheel-cache-wxb107kh\wheels\35\3e\df\3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip

   ---------------------------------------- 0/2 [ftfy]
   ---------------------------------------- 0/2 [ftfy]
   ---------------------------------------- 0/2 [ftfy]
   ---------------------------------------- 0/2 [ftfy]
   -------------------------------

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\ZainabFR\AppData\Local\Temp\pip-req-build-v7lx0zrs'
  DEPRECATION: Building 'clip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'clip'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [8]:
%pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 932.9 kB/s eta 0:00:12
   -- ------------------------------------- 0.8/11.1 MB 1.2 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.1 MB 1.3 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.1 MB 1.3 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.1 MB 1.3 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.3 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.1 MB 1.3 MB/s eta 0:00:07
   --------- ------------------------------ 2.6/11.1 MB 1.4 MB/s eta 0:00:07
   ------------ --------------------------- 3.4/11.1 MB 1.5 MB/s eta 0:00:05
   -------------- ------------------------- 3.9/11.1 MB 1.6 MB/s eta 0:00:05
   ---------------- ----------------------- 4.5/11.1 MB 1.7 MB/s eta 0:00:04
   --------

In [9]:
from flask import Flask, request, jsonify
import torch
import torchvision.transforms as transforms
import clip
import os
import requests
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

# Load API Key from .env
load_dotenv("Unsplash.env")
UNSPLASH_ACCESS_KEY = os.getenv("UNSPLASH_ACCESS_KEY")

# Initialize Flask app
app = Flask(__name__)

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def fetch_images(query="technology", num_images=5):
    """Fetch images from Unsplash based on a search query."""
    url = f"https://api.unsplash.com/photos/random?query={query}&count={num_images}&client_id={UNSPLASH_ACCESS_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        images = response.json()
        image_data = []
        
        for img in images:
            img_url = img["urls"]["regular"]
            img_id = img["id"]
            image_data.append({"id": img_id, "url": img_url})

        return image_data
    else:
        return {"error": f"Error fetching images: {response.status_code}"}

def process_images(image_list):
    """Download and encode images using CLIP."""
    image_embeddings = {}

    for img in image_list:
        img_url = img["url"]
        response = requests.get(img_url, stream=True)
        
        if response.status_code == 200:
            image = Image.open(response.raw).convert("RGB")
            image = preprocess(image).unsqueeze(0).to(device)

            with torch.no_grad():
                embedding = model.encode_image(image).cpu().numpy().flatten()
            
            image_embeddings[img["id"]] = embedding

    return image_embeddings

def recommend_images(query, image_list, top_k=3):
    """Find the most similar images to a given text query using CLIP embeddings."""
    # Encode the text query
    text = clip.tokenize([query]).to(device)
    with torch.no_grad():
        text_embedding = model.encode_text(text).cpu().numpy().flatten()

    # Compute similarity scores
    image_embeddings = process_images(image_list)
    similarities = {
        img_id: cosine_similarity([text_embedding], [embedding])[0][0]
        for img_id, embedding in image_embeddings.items()
    }

    # Get top K results
    sorted_results = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # Retrieve recommended images
    recommended_images = [img for img in image_list if img["id"] in dict(sorted_results)]

    return recommended_images

@app.route('/search', methods=['GET'])
def search_images():
    """API endpoint to fetch and recommend images based on a search query."""
    query = request.args.get("query", "technology")
    num_images = int(request.args.get("num_images", 5))
    
    # Fetch images from Unsplash
    image_list = fetch_images(query, num_images)
    
    # If an error occurred while fetching images
    if isinstance(image_list, dict) and "error" in image_list:
        return jsonify(image_list)

    # Find similar images using CLIP
    recommended = recommend_images(query, image_list, top_k=3)

    return jsonify(recommended)

if __name__ == '__main__':
    app.run(debug=True)


100%|███████████████████████████████████████| 338M/338M [02:56<00:00, 2.00MiB/s]


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\ZainabFR\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
